In [1]:
import mne

In [2]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import signal
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import random

In [3]:
data =pd.read_csv(r"C:\Users\jogip\Desktop\NIT\Data\labels.csv")

In [4]:
data.head(10)

,experiment_code,arousal,valence,emotion
0,SUB01TST04,8,3,fear
1,SUB01TST03,3,1,sad
2,SUB01TST02,5,5,neutral
3,SUB01VID25,8,7,surprise
4,SUB01VID24,4,4,sad
5,SUB01VID23,3,3,sad
6,SUB01VID21,7,1,angry
7,SUB01VID20,8,1,disgust
8,SUB01VID19,2,5,neutral
9,SUB01VID18,8,2,disgust


In [5]:
from glob import glob
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
root_path =r"C:\Users\jogip\Desktop\NIT\Data"

In [7]:
label_filepath= root_path+'\labels.csv'
label_df = pd.read_csv(label_filepath)

In [8]:
def get_label( file_path):

  # Extract the filename from the absolute path
  filename = os.path.basename(file_path)

# Split the filename into parts using "_" as the delimiter
  filename_parts = filename.split("_")


# Specify the experiment code you are interested in
  target_experiment_code = filename_parts[0]
  # print('*****',target_experiment_code)
# Use loc to retrieve the arousal value for the specified experiment code
  arousal_value = label_df.loc[label_df['experiment_code'] == target_experiment_code, 'arousal' ].values

  if arousal_value < 4:
      return 0
  else:
      return 1 #arousal_value

In [9]:
root_path

'C:\\Users\\jogip\\Desktop\\NIT\\Data'

In [10]:
eeg_folder_path = root_path+'\Emotiv'+'\Emotiv'

import os

# Specify the root folder
# root_folder = '/path/to/your/root/folder'
fif_file_list =[]
# Walk through the root folder and its subfolders
for folder_path, subfolders, files in os.walk(eeg_folder_path):
    # print(f"Current folder: {folder_path}")
    fif_files = [os.path.join(folder_path, file) for file in files if file.endswith('.fif')]
    fif_file_list.extend(fif_files)
    # print(fif_files)
    # Print all files in the current subfolder
    # for file in fif_files:
    #     file_path = os.path.join(folder_path, file)
    #     print(f"  File: {file_path}")




In [11]:
def read_data(file_path):
    raw = mne.io.read_raw_fif(file_path, preload=True)

    #preprocess funtions
    raw.set_eeg_reference()
    raw.filter(l_freq=1,h_freq=45)
    return raw.get_data()

    # return epochs #trials,channel,length

In [12]:
len(fif_file_list)

487

In [13]:
raw_list = []
labels_list = []

# Iterate through the EEG epoch files in the folder
for file_path in fif_file_list:
    # Load each epoch file using MNE
    label=get_label(file_path)
    labels_list.append(label)
    raw = read_data(file_path)
    raw_list.append(raw)

Opening raw data file C:\Users\jogip\Desktop\NIT\Data\Emotiv\Emotiv\SUB01\SUB01VID01_eeg.fif...
    Range : 0 ... 15616 =      0.000 ...    61.000 secs
Ready.
Reading 0 ... 15616  =      0.000 ...    61.000 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 845 samples (3.301 s)

Opening raw data file C:\Users\jogip\Desktop\NIT\Data\Emotiv\Emotiv\SUB01\SUB01VID02_eeg.fif...
    Range : 0 ...

In [14]:
fif_file_list

['C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID01_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID02_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID03_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID04_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID05_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID07_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID08_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID09_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID10_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID11_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID12_eeg.fif',
 'C:\\Users\\jogip\\Desktop\\NIT\\Data\\Emotiv\\Emotiv\\SUB01\\SUB01VID13_ee

In [15]:
type(raw_list)

list

In [16]:
import numpy as np
from mne.io import read_raw_fif
from scipy.fft import fft

# Define a function to apply Fourier transform to each epoch
def apply_fourier_transform(data):
    # Assuming data has shape (channels, samples)
    transformed_data = np.abs(fft(data, axis=1))  # Calculate magnitude of the Fourier transform
    return transformed_data

raw_list_data = []
labels_list = []

# Iterate through the EEG epoch files in the folder
for file_path in fif_file_list:
    # Load each epoch file using MNE
    label = get_label(file_path)
    labels_list.append(label)
    
    # Read raw EEG data
    raw = read_raw_fif(file_path, preload=True)
    data, _ = raw[:, :]  # Get the EEG data as a NumPy array
    
    # Apply Fourier transform to the raw EEG data
    transformed_data = apply_fourier_transform(data)
    
    # Append the transformed data to the raw_list
    raw_list_data.append(transformed_data)


Opening raw data file C:\Users\jogip\Desktop\NIT\Data\Emotiv\Emotiv\SUB01\SUB01VID01_eeg.fif...
    Range : 0 ... 15616 =      0.000 ...    61.000 secs
Ready.
Reading 0 ... 15616  =      0.000 ...    61.000 secs...
Opening raw data file C:\Users\jogip\Desktop\NIT\Data\Emotiv\Emotiv\SUB01\SUB01VID02_eeg.fif...
    Range : 0 ... 30976 =      0.000 ...   121.000 secs
Ready.
Reading 0 ... 30976  =      0.000 ...   121.000 secs...
Opening raw data file C:\Users\jogip\Desktop\NIT\Data\Emotiv\Emotiv\SUB01\SUB01VID03_eeg.fif...
    Range : 0 ... 30976 =      0.000 ...   121.000 secs
Ready.
Reading 0 ... 30976  =      0.000 ...   121.000 secs...
Opening raw data file C:\Users\jogip\Desktop\NIT\Data\Emotiv\Emotiv\SUB01\SUB01VID04_eeg.fif...
    Range : 0 ... 16128 =      0.000 ...    63.000 secs
Ready.
Reading 0 ... 16128  =      0.000 ...    63.000 secs...
Opening raw data file C:\Users\jogip\Desktop\NIT\Data\Emotiv\Emotiv\SUB01\SUB01VID05_eeg.fif...
    Range : 0 ... 46336 =      0.000 ...   1

In [17]:
def make_windowing(X,Y):
    X_new = []
    Y_new = []
    npt = 128 # Window length
    stride = 128 # stride lenth
    ctr = 0
    for i in range(0,X.shape[0]):
        y = Y[i]
        a= X[i,:,:]
        a = a.transpose()
        a.shape
        val = 0
        kd=len(a)
        while val<=(len(a)-npt):
            x = a[val:val+npt,:]

            X_new.append(x.T)
            Y_new.append(y)
            val = val+stride

    return np.array(X_new),np.array(Y_new)

In [18]:
from scipy import stats
def mean(data):
    return np.mean(data,axis=-1)

def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1)

def var(data):
      return np.var(data,axis=-1)

def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)


def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=-1))

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)


def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

def concatenate_features(data):
    return np.concatenate((mean(data),
                           # std(data),
                          # ptp(data),
                          #  var(data),
                          #  minim(data),
                          #  maxim(data),
                          #  argminim(data),
                          #  argmaxim(data),
                            mean_square(data),
                            rms(data),
                         # abs_diffs_signal(data),
                           skewness(data),
                           kurtosis(data)
                          ), axis=-1)

In [19]:
from tqdm import tqdm
features=[]
for raw in tqdm(raw_list_data):
  features.append(concatenate_features(raw))
features=np.array(features)
labels = np.array(labels_list)

100%|██████████| 487/487 [00:17<00:00, 27.49it/s]


In [21]:
features

array([[ 5047.93827961,  5110.19431216,  5017.89685914, ...,
        15611.40278553, 15611.30884809, 15611.4053462 ],
       [ 5323.85401319,  5402.83032023,  5281.0118263 , ...,
        30965.78866122, 30970.65172968, 30970.15100956],
       [ 5404.70187639,  5517.26241644,  5363.57149659, ...,
        30969.6813329 , 30970.61094727, 30969.61535337],
       ...,
       [ 5726.16041021,  5564.93727017,  5621.07474649, ...,
        30970.29961533, 30969.64703521, 30968.64840161],
       [ 5775.70813024,  5651.62740332,  5522.78644074, ...,
        30970.28343706, 30969.34222524, 30968.65715589],
       [ 5683.47875584,  5567.24151406,  5399.6245996 , ...,
        30970.58641038, 30970.47616347, 30969.61192601]])

In [22]:
labels

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# Create a preprocessing pipeline with StandardScaler and PCA
pipeline = make_pipeline(StandardScaler(), PCA(n_components=35))

# Fit and transform the training data
X_train_preprocessed = pipeline.fit_transform(X_train)

# Transform the test data
X_test_preprocessed = pipeline.transform(X_test)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=35, random_state=42)
clf.fit(X_train_preprocessed, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_preprocessed)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)

precision = metrics.precision_score(y_test, y_pred, average='macro')
recall = metrics.recall_score(y_test, y_pred, average='macro')
f1 = metrics.f1_score(y_test, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


Accuracy: 0.673469387755102
Precision: 0.4636363636363636
Recall: 0.48290598290598286
F1-Score: 0.4555555555555555


In [25]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assuming 'features' is your input data and 'labels' is the target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# Create a preprocessing pipeline with StandardScaler and PCA
pipeline = make_pipeline(StandardScaler(), PCA(n_components=35))

# Fit and transform the training data
X_train_preprocessed = pipeline.fit_transform(X_train)

# Transform the test data
X_test_preprocessed = pipeline.transform(X_test)

# Create a simple neural network model with Keras
def create_neural_network(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Get the number of features after PCA
input_dim = X_train_preprocessed.shape[1]

# Create the neural network model
model = create_neural_network(input_dim)

# Train the model
model.fit(X_train_preprocessed, y_train, epochs=5, batch_size=32, validation_split=0.1)

# Make predictions on the test set
y_pred_probs = model.predict(X_test_preprocessed)
y_pred = np.round(y_pred_probs)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')




Epoch 1/5


13/13 [==============================] - 1s 19ms/step - loss: 0.7611 - accuracy: 0.4670 - val_loss: 0.6693 - val_accuracy: 0.6364
Epoch 2/5
13/13 [==============================] - 0s 4ms/step - loss: 0.6195 - accuracy: 0.6853 - val_loss: 0.6200 - val_accuracy: 0.7273
Epoch 3/5
13/13 [==============================] - 0s 5ms/step - loss: 0.5838 - accuracy: 0.7030 - val_loss: 0.6169 - val_accuracy: 0.7500
Epoch 4/5
13/13 [==============================] - 0s 4ms/step - loss: 0.5717 - accuracy: 0.7005 - val_loss: 0.6475 - val_accuracy: 0.7273
Epoch 5/5
2/2 [==============================] - 0s 4ms/step
Accuracy: 0.7346938775510204


In [42]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Generate synthetic data
X, y = make_classification(n_samples=1000, n_features=50, n_classes=2, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Create a preprocessing pipeline with StandardScaler and PCA
pipeline = make_pipeline(StandardScaler(), PCA(n_components=35))

# Fit and transform the training data
X_train_preprocessed = pipeline.fit_transform(X_train)

# Transform the test data
X_test_preprocessed = pipeline.transform(X_test)

# Get the number of features after PCA
input_dim = X_train_preprocessed.shape[1]

# Create the neural network model
model = create_neural_network(input_dim)

# Train the model
model.fit(X_train_preprocessed, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Make predictions on the test set
y_pred_probs = model.predict(X_test_preprocessed)
y_pred = np.round(y_pred_probs)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Epoch 1/10
26/26 [==============================] - 1s 7ms/step - loss: 0.7137 - accuracy: 0.5457 - val_loss: 0.6572 - val_accuracy: 0.6000
Epoch 2/10
26/26 [==============================] - 0s 3ms/step - loss: 0.5886 - accuracy: 0.7531 - val_loss: 0.5954 - val_accuracy: 0.7444
Epoch 3/10
26/26 [==============================] - 0s 3ms/step - loss: 0.4959 - accuracy: 0.8235 - val_loss: 0.5384 - val_accuracy: 0.7333
Epoch 4/10
26/26 [==============================] - 0s 3ms/step - loss: 0.4164 - accuracy: 0.8420 - val_loss: 0.4984 - val_accuracy: 0.7778
Epoch 5/10
26/26 [==============================] - 0s 3ms/step - loss: 0.3585 - accuracy: 0.8617 - val_loss: 0.4796 - val_accuracy: 0.8222
Epoch 6/10
26/26 [==============================] - 0s 2ms/step - loss: 0.3204 - accuracy: 0.8704 - val_loss: 0.4823 - val_accuracy: 0.8111
Epoch 7/10
26/26 [==============================] - 0s 2ms/step - loss: 0.2950 - accuracy: 0.8864 - val_loss: 0.4833 - val_accuracy: 0.8111
Epoch 8/10
26/26 [==

In [43]:
# Make predictions on the training set
y_train_pred_probs = model.predict(X_train_preprocessed)
y_train_pred = np.round(y_train_pred_probs)

# Evaluate the classifier on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f'Training Accuracy: {train_accuracy}')


 1/29 [>.............................] - ETA: 0s

29/29 [==============================] - 0s 2ms/step
Training Accuracy: 0.9077777777777778


In [46]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Generate synthetic data
X, y = make_classification(n_samples=1000, n_features=50, n_classes=2, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Create a preprocessing pipeline with StandardScaler and PCA
pipeline = make_pipeline(StandardScaler(), PCA(n_components=35))

# Fit and transform the training data
X_train_preprocessed = pipeline.fit_transform(X_train)

# Get the number of features after PCA
input_dim = X_train_preprocessed.shape[1]

# Create the neural network model
model = Sequential()
model.add(Dense(64, input_dim=input_dim, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_preprocessed, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Make predictions on the training set
y_train_pred_probs = model.predict(X_train_preprocessed)
y_train_pred = np.round(y_train_pred_probs)

# Evaluate the classifier on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f'Training Accuracy: {train_accuracy}')

# Save the trained model
model.save('neural_network_model.h5')


Epoch 1/10
26/26 [==============================] - 1s 10ms/step - loss: 0.6864 - accuracy: 0.5889 - val_loss: 0.6095 - val_accuracy: 0.7000
Epoch 2/10
26/26 [==============================] - 0s 4ms/step - loss: 0.5450 - accuracy: 0.7506 - val_loss: 0.5353 - val_accuracy: 0.8111
Epoch 3/10
26/26 [==============================] - 0s 3ms/step - loss: 0.4547 - accuracy: 0.8099 - val_loss: 0.4893 - val_accuracy: 0.8444
Epoch 4/10
26/26 [==============================] - 0s 3ms/step - loss: 0.3929 - accuracy: 0.8420 - val_loss: 0.4662 - val_accuracy: 0.8667
Epoch 5/10
26/26 [==============================] - 0s 3ms/step - loss: 0.3538 - accuracy: 0.8630 - val_loss: 0.4538 - val_accuracy: 0.8667
Epoch 6/10
26/26 [==============================] - 0s 3ms/step - loss: 0.3245 - accuracy: 0.8778 - val_loss: 0.4582 - val_accuracy: 0.8556
Epoch 7/10
26/26 [==============================] - 0s 5ms/step - loss: 0.3046 - accuracy: 0.8790 - val_loss: 0.4590 - val_accuracy: 0.8556
Epoch 8/10
26/26 [=

c:\Users\jogip\deskt\envs\nit\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 0s 1ms/step
Training Accuracy: 0.9066666666666666


In [48]:
import mne
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Function to create the neural network model
def create_neural_network(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Function to preprocess EEG data
def preprocess_eeg_data(raw_data, scaler, pca):
    # Apply Fourier transform
    frequencies, spectrum = mne.time_frequency.psd_welch(raw_data, fmin=1, fmax=45, n_fft=2048)
    # Take the mean spectrum across channels as a feature vector
    feature_vector = np.mean(spectrum, axis=0)
    # Reshape to (1, n_features) as the model expects a batch of data
    feature_vector_reshaped = feature_vector.reshape(1, -1)
    # Preprocess using the provided scaler and PCA
    preprocessed_data = pca.transform(scaler.transform(feature_vector_reshaped))
    return preprocessed_data

# Load EEG data from file
file_path = r'C:\Users\jogip\Desktop\NIT\Data\Emotiv\Emotiv\SUB01\SUB01VID04_eeg.set'
raw_data = mne.io.read_raw_eeglab(file_path, preload=True)

# Create a preprocessing pipeline with StandardScaler and PCA
pipeline = make_pipeline(StandardScaler(), PCA(n_components=35))

# Preprocess the EEG data
X_preprocessed = preprocess_eeg_data(raw_data, pipeline.named_steps['standardscaler'], pipeline.named_steps['pca'])

# Get the number of features after PCA
input_dim = X_preprocessed.shape[1]

# Create the neural network model
model = create_neural_network(input_dim)

# Load the trained model weights
model.load_weights('neural_network_model.h5')

# Make predictions on the preprocessed EEG data
output_probs = model.predict(X_preprocessed)
output = np.round(output_probs)

# Print the output
print("Model Output:", output)


AttributeError: No mne.time_frequency attribute psd_welch

In [49]:
import mne
print(mne.__version__)


1.6.0


In [ ]:
pip install --upgrade mne
